In [1]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
import csv
import numpy as np
import pandas as pd
from scipy import sparse
import pickle
import os
os.chdir('G:/Mi unidad/SOCOFAR/0.-Recomendadores-Oficial')
import Pipeline_Recomendaciones
class_reco = Pipeline_Recomendaciones.clase_recomendadores()
os.chdir('G:/Mi unidad/SOCOFAR/0.-Recomendadores-Oficial/Archivos')

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### **Train Model**

In [2]:
#CARGA DATOS TRAIN
class_reco.carga_data(base_transacciones='Recomendaciones_PDC_OCT23/SR_BASE_FINAL_PROD_OCT23.csv',recomendador='cliente_antiguo'
                      ,user_features=['GENERO'] #,'GSE'
                      ,item_features=['MARCA']
                      ,diccionario='diccionario_productos.csv')

In [3]:
#interacciones train
class_reco.interaction_feature_matrix(recomendador='cliente_antiguo'
                                     ,name_output_interaction_matrix='INTERACTIONS_DEV.npz'
                                     ,model_feature=True #False == NO DEMOGRAFICAS
                                     )

In [4]:
class_reco.train_cliente_antiguo(name_output_model='RECOS_DEV.pkl'
                                ,name_interaction_matrix='INTERACTIONS_DEV.npz'
                                ,model_feature=True #False == NO DEMOGRAFICAS
                                ,epoch=20)

#### **TEST MODEL**

In [12]:
dataset_test=pd.read_csv('Recomendaciones_PDC_NOV23_ENE24/SR_BASE_FINAL_PROD_NOV23_ENE24.csv',sep=';')
dataset_test['RUT_CLIENTE']=((dataset_test['SK']+1599)/2).astype(int)
products_train=pd.read_csv('products_cliente_antiguo.csv')
customers_train=pd.read_csv('customers_cliente_antiguo.csv')
dataset_test=dataset_test[dataset_test.CODIGO_PRODUCTO.isin(products_train.products)]
dataset_test=dataset_test[dataset_test.RUT_CLIENTE.isin(customers_train.customers)]
dataset_test.to_csv('Recomendaciones_PDC_NOV23_ENE24/SR_BASE_FINAL_PROD_NOV23_ENE24_ON_TRAIN.csv',sep=';')

In [2]:
#CARGA DATOS TEST
class_reco.carga_data(base_transacciones='Recomendaciones_PDC_NOV23_ENE24/SR_BASE_FINAL_PROD_NOV23_ENE24_ON_TRAIN.csv',recomendador='cliente_antiguo'
                      ,user_features=['GENERO'] #,'GSE'
                      ,item_features=['MARCA']
                      ,diccionario='diccionario_productos.csv'
                      ,test=True)

In [6]:
#interacciones test
class_reco.interaction_feature_matrix(recomendador='cliente_antiguo'
                                     ,name_output_interaction_matrix='INTERACTIONS_TEST.npz'
                                     ,model_feature=True #False == NO DEMOGRAFICAS
                                     )

test=class_reco.df_coo

In [ ]:
test = sparse.load_npz('INTERACTIONS_TEST.npz')

In [ ]:
with open('G:/Mi unidad/SOCOFAR/0.-Recomendadores-Oficial/Archivos/RECOS_DEV.pkl', "rb") as file:
            class_reco.model=pickle.load(file)

In [ ]:
user_features_matrix = sparse.load_npz('user_features_INTERACTIONS_TEST.npz')
item_features_matrix = sparse.load_npz('item_features_INTERACTIONS_TEST.npz')

In [ ]:
k = 3
precision = precision_at_k(model=class_reco.model, test_interactions=test 
                           ,user_features=user_features_matrix
                           ,item_features=item_features_matrix
                           ,k=k).mean()
recall = recall_at_k(class_reco.model, test_interactions=test
                     ,user_features=user_features_matrix
                     ,item_features=item_features_matrix
                     ,k=k).mean()

print("Precisión en el top", k, ":", precision)
print("Recall en el top", k, ":", recall)

In [ ]:
k = 4
precision = precision_at_k(model=class_reco.model, test_interactions=test 
                           ,user_features=user_features_matrix
                           ,item_features=item_features_matrix
                           ,k=k).mean()
recall = recall_at_k(class_reco.model, test_interactions=test
                     ,user_features=user_features_matrix
                     ,item_features=item_features_matrix
                     ,k=k).mean()

print("Precisión en el top", k, ":", precision)
print("Recall en el top", k, ":", recall)

Precisión en el top 4 : 0.051626272
Recall en el top 4 : 0.03163892687569442


In [ ]:
k = 5
precision = precision_at_k(model=class_reco.model, test_interactions=test 
                           ,user_features=user_features_matrix
                           ,item_features=item_features_matrix
                           ,k=k).mean()
recall = recall_at_k(class_reco.model, test_interactions=test
                     ,user_features=user_features_matrix
                     ,item_features=item_features_matrix
                     ,k=k).mean()

print("Precisión en el top", k, ":", precision)
print("Recall en el top", k, ":", recall)

Precisión en el top 5 : 0.047631975
Recall en el top 5 : 0.03754599043509295


In [ ]:
# k = 3
# Precisión en el top 3 : 0.058377754
# Recall en el top 3 : 0.026069278704120593

# k = 4
# Precisión en el top 4 : 0.051626272
# Recall en el top 4 : 0.03163892687569442

# k = 5
# Precisión en el top 5 : 0.047631975
# Recall en el top 5 : 0.03754599043509295